## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/6/16

### Libaries

In [1]:
import pandas as pd#读取csv文件的库
import numpy as np#进行矩阵运算和科学计算的库
#model lgb分类模型,日志评估,早停防止过拟合
from  lightgbm import LGBMClassifier,log_evaluation,early_stopping
#KFold是直接分成k折,StratifiedKFold还要考虑每种类别的占比
from sklearn.model_selection import StratifiedGroupKFold
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

### Data

In [2]:
path="/kaggle/input/2024kedaxunfeimentalhealth/2024xunfeimentalhealth/"
train_label=pd.read_csv(path+"training_data/train_label.csv",encoding='gbk')
train_label.columns=['文件名','label']
print(f"len(train_label):{len(train_label)}")
test=pd.read_csv(path+"example.csv",encoding='gbk',header=None)
test.columns=['文件名','label']
print(f"len(test):{len(test)}")
test.head()

len(train_label):40
len(test):39


,文件名,label
0,data0033,0
1,data0129,0
2,data0157,0
3,data0172,0
4,data0179,0


### Feature engineer

In [3]:
def get_time(time):
    return 3600*int(time[0:2])+60*int(time[3:5])+int(time[6:8])-3600*6
def get_data(dataidxs=train_label['文件名'].values,mode='training_data'):
    datas=[]
    for idx in dataidxs:
        #acc里是某个时间recording_time的Motion_dataX,Motion_dataY,Motion_dataZ
        acc=pd.read_csv(path+mode+f"/{idx}/ACC.csv")
        #gsr是某个时间recording_time的gsr 皮肤电导
        gsr=pd.read_csv(path+mode+f"/{idx}/GSR.csv")
        #ppg是某个时间recording_time的ppg 脉搏波
        ppg=pd.read_csv(path+mode+f"/{idx}/PPG.csv")
        #print(f"len(acc):{len(acc)},len(gsr):{len(gsr)},len(ppg):{len(ppg)}")
        #其中GSR显示为0.0061为缺失值，表示实验对象未正确采集数据
        gsr=gsr.replace(0.0061,np.nan)
        acc=acc.groupby('recording_time')[['Motion_dataX','Motion_dataY','Motion_dataZ']].mean().reset_index()
        gsr=gsr.groupby('recording_time')['GSR'].mean().reset_index()
        ppg=ppg.groupby('recording_time')['PPG'].mean().reset_index()
        #print(f"len(acc):{len(acc)},len(gsr):{len(gsr)},len(ppg):{len(ppg)}")
        #3表合一
        df=acc.merge(gsr,on='recording_time',how='left')
        df=df.merge(ppg,on='recording_time',how='left')
        df['recording_time']=df['recording_time'].apply(lambda x:get_time(x))

        print("gap_feature") 
        cols=['Motion_dataX', 'Motion_dataY', 'Motion_dataZ', 'GSR','PPG']
        gaps=[1,2,3,7,15,30,60,120,300,500,1000]
        for col in cols:
            for gap in gaps:
                df[f"{col}_shift{gap}"]=df[col].shift(gap)
                df[f"{col}_gap{gap}"]=df[col]-df[f"{col}_shift{gap}"]
        df.dropna(inplace=True)
        margin=60*5#5分钟
        for i in range(0,len(df),margin//3):
            #第几号数据,开始的时间
            datas.append([idx,df['recording_time'].values[i]]+
                        list(df.drop(['recording_time'],axis=1).values[i:i+margin].mean(axis=0))+
                        list(df.drop(['recording_time'],axis=1).values[i:i+margin].min(axis=0))+
                        list(df.drop(['recording_time'],axis=1).values[i:i+margin].max(axis=0))+
                        list(df.drop(['recording_time'],axis=1).values[i:i+margin].std(axis=0)) 
                        )
    datas=pd.DataFrame(datas)
    datas.columns=['文件名','recording_time']+\
                  [f'mean_{col}'for col in list(df.drop(['recording_time'],axis=1).columns)]+\
                  [f'min_{col}'for col in list(df.drop(['recording_time'],axis=1).columns)]+\
                  [f'max_{col}'for col in list(df.drop(['recording_time'],axis=1).columns)]+\
                  [f'std_{col}'for col in list(df.drop(['recording_time'],axis=1).columns)]
    cols=['Motion_dataX', 'Motion_dataY', 'Motion_dataZ', 'GSR','PPG']
    for col in cols:
        datas[f'ptp_{col}']=datas[f"max_{col}"]-datas[f"min_{col}"]
    return datas
        
train_data=get_data(dataidxs=train_label['文件名'].values,mode='training_data')
train_data=train_data.merge(train_label,on='文件名',how='left')
print(f"len(train_data):{len(train_data)}")
test_data=get_data(dataidxs=test['文件名'].values,mode='test_data')
print(f"len(test_data):{len(test_data)}")
test_data.head()

gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
len(train_data):14046
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
gap_feature
len(test_data):12927


,文件名,recording_time,mean_Motion_dataX,mean_Motion_dataY,mean_Motion_dataZ,mean_GSR,mean_PPG,mean_Motion_dataX_shift1,mean_Motion_dataX_gap1,mean_Motion_dataX_shift2,...,std_PPG_gap300,std_PPG_shift500,std_PPG_gap500,std_PPG_shift1000,std_PPG_gap1000,ptp_Motion_dataX,ptp_Motion_dataY,ptp_Motion_dataZ,ptp_GSR,ptp_PPG
0,data0033,14261,5.786155,5.218630,-1.415284,0.402465,49.680462,5.801548,-0.015393,5.859371,...,12.163027,6.548519,11.467684,16.337404,19.601221,19.680869,12.341875,16.411770,0.168585,30.007705
1,data0033,14361,5.270226,5.189863,-0.591445,0.404259,48.403355,5.266878,0.003348,5.265786,...,11.341858,6.355649,11.514272,3.964593,9.143690,20.372085,12.194851,17.835713,0.175948,40.767401
2,data0033,14461,4.965142,6.852190,0.064810,0.397592,45.109231,4.980873,-0.015731,4.994177,...,8.984200,6.098927,11.169338,5.585473,11.665804,20.372085,11.188652,17.835713,0.175948,41.317388
3,data0033,14561,5.123464,7.311587,0.809634,0.386450,44.538273,5.112332,0.011132,5.067181,...,10.458849,5.526373,11.183968,7.853034,14.492500,13.367214,12.949470,19.310139,0.203488,41.317388
4,data0033,14661,6.105747,6.181549,-0.667871,0.354803,41.144867,6.091235,0.014512,6.100595,...,10.794520,7.962128,9.594974,7.899100,13.437630,16.244170,12.971721,16.040083,0.205770,30.148193


### Model training

In [4]:
choose_cols=list(test_data.drop(['文件名'],axis=1).columns)
def fit_and_predict(train_feats=train_data,test_feats=test_data,
                    model=None,num_folds=10,name='lgb'):
    X=train_feats[choose_cols].copy()
    y=train_feats['label'].copy()
    group=train_feats['文件名'].values
    oof_preds=np.zeros(len(X))
    
    test_X=test_feats[choose_cols].copy()
    test_preds=np.zeros((num_folds,len(test_X)))
    
    useless_cols=[]
    
    #10折交叉验证
    sgkf = StratifiedGroupKFold(n_splits=num_folds, shuffle=False)
    for fold, (train_index, valid_index) in (enumerate(sgkf.split(X,y,group))):
        print(f"name {name},fold:{fold}")

        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        model.fit(X_train,y_train,eval_set=[(X_valid, y_valid)],
                     callbacks=[early_stopping(100)],
                     ) 
        oof_preds[valid_index]=model.predict(X_valid)
        test_preds[fold]=model.predict_proba(test_X)[:,1]
        
        feature_importance=list(model.feature_importances_)
        for i in range(len(choose_cols)):
            if (feature_importance[i]==0) and (choose_cols[i] not in useless_cols):
                useless_cols.append(choose_cols[i])
    print(f"useless_cols:{useless_cols}")
    print(f"accuracy:{np.mean( y.values== (oof_preds>=0.5).astype(int) )}" )
        
    test_preds=test_preds.mean(axis=0)

    return test_preds

lgb_params={
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 12,
    "learning_rate": 0.05,
    "n_estimators": 560,
    "colsample_bytree": 0.9,
    "colsample_bynode": 0.9,
    "verbose": -1,
    "random_state": 2024,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':64,
    "verbose": -1,
    "max_bin":255,
    }
test_preds=fit_and_predict(model=LGBMClassifier(**lgb_params),num_folds=10,name='lgb')

name lgb,fold:0
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[241]	valid_0's auc: 0.840134
name lgb,fold:1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5]	valid_0's auc: 0.921978
name lgb,fold:2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[251]	valid_0's auc: 0.902589
name lgb,fold:3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[204]	valid_0's auc: 0.866408
name lgb,fold:4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[136]	valid_0's auc: 0.751387
name lgb,fold:5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[245]	valid_0's auc: 0.825597
name lgb,fold:6
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	valid_0's auc: 0.973881
na

### Submission

In [5]:
test_data['label']=test_preds
test_data=test_data.groupby(['文件名'])['label'].mean().reset_index()
test_data['label']=(test_data['label']>=0.5).astype(int)
test_data.to_csv('baseline.csv',index=None,header=None)
test_data.head()

,文件名,label
0,data0033,0
1,data0129,1
2,data0157,0
3,data0172,0
4,data0179,1
